In [1]:
# Train the CNN Emotion Regression Model with audio data#
# Last editted by Pu Zeng, 18/10/2023 #
# The result is really bad because we only use a small portion as the toy example

In [40]:
import os
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
# import mido
import string
import numpy as np
# from utilis import get_pianoroll_data
import pickle
import pandas as pd
from tqdm import tqdm
import os
from sklearn.model_selection import KFold
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

from torch.utils.data import TensorDataset
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import Dataset, DataLoader,TensorDataset,random_split,SubsetRandomSampler, ConcatDataset
from sklearn import preprocessing
import torch
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


In [41]:
!python ../Data_Preprocessing/MP3_Emotion.py ../Toy_Dataset/MP3_Emotion ./Processed_Data


  3%|▎         | 23/744 [00:12<01:09, 10.32it/s]C:\Milestone 2\Milestone 2\696-Milestone\Data_Preprocessing\MP3_Emotion.py:19: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(audio_path)
D:\programing\anaconda3\lib\site-packages\librosa\core\audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)

100%|██████████| 744/744 [00:12<00:00, 59.74it/s]  


In [42]:
# Read Data
file=open("./Processed_data/mfcc_emotion_dataset","rb")
music_data = pickle.load(file) 
file.close()

In [43]:
music_data[0][0].shape

(20, 1940)

In [45]:
dim=np.min([x[0].shape[1] for x in music_data])
dim=1922

In [46]:
from tqdm import tqdm
X = []
y = []
for m in music_data:
    X.append(m[0].T[0:1922,:])
    y.append([m[1], m[2]])
X = np.array(X)

In [47]:
#Preprocessing Data
from sklearn import preprocessing
import torch

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) # First split to avoid data leakage
m = np.vstack(X_train).mean()
std = np.vstack(X_train).std()
X_train = (X_train-m)/std
X_test = (X_test-m)/std

import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

from torch.utils.data import TensorDataset

X_train = torch.as_tensor(X_train, dtype=torch.float) # an alternative to torch.from_numpy
y_train = torch.as_tensor(y_train, dtype=torch.float)
X_test = torch.as_tensor(X_test, dtype=torch.float)
y_test = torch.as_tensor(y_test, dtype=torch.float)

train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

from torch.utils.data import Dataset, DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=128, shuffle=False)


In [48]:
#Define CNN Regression Model

class CNNModel(nn.Module):
    def __init__(self, hiddenSize, outChannels, dropoutRate, activate):
        super().__init__()
        self.outChannels = outChannels
        self.activate = nn.Sigmoid() if activate == "Sigmoid" else nn.ReLU()
        self.conv1 = nn.Conv2d(1, 24, (10,1))
        self.pool = nn.MaxPool2d((3, 1))
        self.conv2 = nn.Conv2d(24, 48, (10,1))
        self.conv3 = nn.Conv2d(48, 96, (10,1))
        self.conv4 = nn.Conv2d(96, 192, (10,2))
        self.conv5 = nn.Conv2d(192, 96, (5,2))
        self.conv6 = nn.Conv2d(96, 96, (5,2))
        self.dense1 = nn.Linear(16*51*2, hiddenSize)
        self.dropout = nn.Dropout(dropoutRate)
        self.dense2 = nn.Linear(hiddenSize, 2)

    def forward(self, x):
        x = self.pool(self.activate(self.conv1(x)))
        x = self.pool(self.activate(self.conv2(x)))
        # print(x.shape)
        x = self.dropout(self.pool(self.activate(self.conv3(x))))
        x = self.pool(self.activate(self.conv4(x)))
        x = self.pool(self.activate(self.conv5(x)))
        
        x = self.activate(self.conv6(x))
        # print(x.shape)
        x = x.view(-1, 16*51*2)
        
        x = self.dropout(self.activate(self.dense1(x)))
        # print(self.dense2(x))
        return self.dense2(x)

# Number of neurons in the first fully-connected layer
hiddenSize = 64
# Number of feature filters in second convolutional layer
numFilters = 25
# Dropout rate
dropoutRate = 0.2
# Activation function
activation = "ReLU"
# Learning rate
learningRate = 0.001
# Momentum for SGD optimizer
momentum = 0.9
# Number of training epochs
numEpochs = 100

In [49]:
#Define Trianing and Validation Function

from tqdm import tqdm
def train_epoch(cnn,device,dataloader,loss_fn,optimizer, m, std):
    cnn.train()
    cnnRunningLoss = 0
    total = 0
    R2 = 0
    for i, (inputs, labels) in enumerate(dataloader, 0):
        optimizer.zero_grad()
        inputs = ((inputs.reshape(-1,1,1922,20) - m)/std).to('cuda')
        labels = labels.to('cuda')
        # Forward propagation
        cnnOutputs = cnn(inputs)

        l2_lambda = 0.05
        l2_reg = torch.tensor(0.).to('cuda')

        for param in cnn.parameters():
            l2_reg += torch.norm(param)

        # Backpropagation
        cnnLoss = criterion(cnnOutputs, labels)+l2_reg*l2_lambda
        cnnLoss.backward()

        # Gradient update
        optimizer.step()

        cnnRunningLoss += cnnLoss.item()
        total += 1
        # if (i+1) % 20 == 0:    # print every 2000 mini-batches
        #     print(cnnLoss)
        R2 += sklearn.metrics.r2_score(labels.to('cpu').detach().numpy(), cnnOutputs.to('cpu').detach().numpy())
    print('Training Loss {}, Trainging R-squared {}'.format(str(cnnRunningLoss/total), str(R2/total)))
    return cnnRunningLoss/total, R2
def valid_epoch(cnn,device,dataloader,loss_fn, m, std):
    cnn.eval()
    totalLoss = 0
    total = 0
    R2 = 0
    for inputs, labels in dataloader:
        inputs = ((inputs.reshape(-1,1,1922,20) - m)/std).to('cuda')
        labels = labels.to('cuda')
        cnnOutputs = cnn(inputs)
        cnnLoss = criterion(cnnOutputs, labels)#+l2_reg*l2_lambda
        # print(cnnLoss)
        totalLoss += cnnLoss.item()
        total +=1
        R2 += sklearn.metrics.r2_score(labels.to('cpu').detach().numpy(), cnnOutputs.to('cpu').detach().numpy())
    print('\nCNN validation R2: {}\n '.format(str(R2/total)))
    cnn.train()
    return totalLoss/total, R2

In [50]:
#K-Fold Validation

import sklearn
X_train_v, X_test, y_train_v, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
X_train_v = torch.as_tensor(X_train_v, dtype=torch.float) # an alternative to torch.from_numpy
y_train_v = torch.as_tensor(y_train_v, dtype=torch.float)
X_test = torch.as_tensor(X_test, dtype=torch.float)
y_test = torch.as_tensor(y_test, dtype=torch.float)
splits=KFold(n_splits=5,shuffle=True,random_state=42)
train_dataset = TensorDataset(X_train_v, y_train_v)
test_dataset = TensorDataset(X_test, y_test)
batch_size=128
models = []
history = {'fold':[], 'train_loss': [],'train_R2':[],'test_R2':[]}

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
for fold, (train_idx,val_idx) in enumerate(splits.split(np.arange(len(X_train_v)))):
    print('Fold {}'.format(fold + 1))
    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(val_idx)
    m = np.vstack(X_train_v[train_idx]).mean()
    std = np.vstack(X_train_v[train_idx]).std()
    # X_train_v[train_idx] = (X_train_v[train_idx]-m)/std
    # X_train_v[val_idx] = (X_train_v[val_idx]-m)/std
    train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=train_sampler)
    test_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=test_sampler)
    
    model = CNNModel(hiddenSize, numFilters, dropoutRate, activation).to(device)
    criterion = nn.MSELoss()
    # optimizer = torch.optim.SGD(list(model.parameters()), lr=learningRate, momentum=momentum)
    optimizer = torch.optim.Adam(list(model.parameters()), lr=learningRate)
    best_test = np.inf
    best_model = None
    
    for epoch in range(numEpochs):
        train_loss, train_R2=train_epoch(model,device,train_loader,criterion,optimizer, m, std)
        test_loss, test_R2=valid_epoch(model,device,test_loader,criterion, m, std)
        history['fold'].append(fold)
        history['train_loss'].append(train_loss)
        history['train_R2'].append(train_R2)
        history['test_R2'].append(test_R2)   
        if test_loss<best_test:
            test_test = test_loss
            best_model = model
    models.append([best_model, m, std])

Fold 1
Training Loss 29.90658187866211, Trainging R-squared -20.58142425952019

CNN validation R2: -28.91828119099564
 
Training Loss 29.000242233276367, Trainging R-squared -19.9699062101743

CNN validation R2: -21.374032672460977
 
Training Loss 21.842742919921875, Trainging R-squared -14.739084170699824

CNN validation R2: -2.928251190892688
 
Training Loss 7.07407283782959, Trainging R-squared -3.3425356984137236

CNN validation R2: -134.68567558663807
 
Training Loss 93.56795501708984, Trainging R-squared -68.88417315293466

CNN validation R2: -1.4739099736473844
 
Training Loss 6.276783466339111, Trainging R-squared -2.4707770138149217

CNN validation R2: -8.99437049523906
 
Training Loss 11.873705863952637, Trainging R-squared -6.350082467035432

CNN validation R2: -19.385898132237234
 
Training Loss 21.059368133544922, Trainging R-squared -13.53648971059288

CNN validation R2: -24.49957537407687
 
Training Loss 25.182788848876953, Trainging R-squared -16.946002120731247

CNN va

In [51]:
# Calculate R2
r2 = []
r2_r = []
r2_v = []
for best_model, m, std in models:
    i = 0
    # best_model=models[1]
    test_loader = DataLoader(test_dataset, batch_size=batch_size)
    best_model.eval()
    for inputs, labels in test_loader:
        inputs = ((inputs.reshape(-1,1,1922,20)-m)/std).to('cuda')
        labels = labels.to('cuda')
        if i == 0:
            cnnOutputs = best_model(inputs).to('cpu')
            l = labels.to('cpu')
        else:
            cnnOutputs = torch.cat((cnnOutputs, best_model(inputs).to('cpu')), 0)
            l = torch.cat((l, labels.to('cpu')), 0)
        i+=1
    r2_v.append(sklearn.metrics.r2_score(l.to('cpu').detach().numpy()[:,1], cnnOutputs.to('cpu').detach().numpy()[:,1]))
    r2_r.append(sklearn.metrics.r2_score(l.to('cpu').detach().numpy()[:,0], cnnOutputs.to('cpu').detach().numpy()[:,0]))
    r2.append(sklearn.metrics.r2_score(l.to('cpu').detach().numpy(), cnnOutputs.to('cpu').detach().numpy()))

In [52]:
print("Average R2: {}, Arousal R2: {}, Valence R2: {}".format(np.mean(r2), np.mean(r2_r), np.mean(r2_v)))

Average R2: -0.5450851804482051, Arousal R2: -0.8435310806255167, Valence R2: -0.2466392802708933
